# Plotting New York State (NYS) Mesonet Data with Archived High-Resolution Rapid Refresh (HRRR) Model Data
This notebook plots NYS Mesonet data from surface-based flux, standard, and profiler site information from 2023-06-06 through 2023-06-07 to analyze the surface energy budget under a severe Canadian wildfire smoke event in the northeast United States. This code was written for a final project in MET 335 (Micrometeorology) at SUNY Oswego during the Fall 2025 semester. </p>
#### **Disclaimer:** To navigate this notebook, please see the *Table of Contents*  </p>
#### **RESOURCES:** </p>
>[Request NYS Mesonet data (please take note of their disclaimer at the top of the webpage](https://www.nysmesonet.org/weather/requestdata) </p>
>[Herbie module documentation](https://herbie.readthedocs.io/en/latest/index.html) </p>
>[HRRR variable list](https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html) </p>
>[**Flux** NYS Mesonet network documentation](https://www.nysmesonet.org/documents/NYSM_Readme_Flux.pdf) </p>
>[**Standard** NYS Mesonet network documentation](https://www.nysmesonet.org/documents/NYSM_Readme_Standard.pdf) </p>
>[**Profiler** NYS Mesonet network documentation](https://nysmesonet.org/documents/NYSM_Readme_Profiler.pdf) </p>
***

## INSTALLATIONS & IMPORTS
Install and import all necessary packages and modules. Also, import necessary datasets
***

In [ ]:
pip install herbie-data

In [ ]:
!pip install cmweather

In [ ]:
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
from herbie import Herbie
import matplotlib as mplt
import matplotlib.cm as cm
import cmweather

In [ ]:
# flux dataset
df=pd.read_csv(r'/home/jovyan/Desktop/Micrometeorology/DATA/Flux_NYSMesonet_202306.csv')
df.rename(columns={
    'stid':'site ID', 
    'FC':'CO2 Flux', 
    'LE': 'HL', 
    'TAU':'Momentum Flux',
    'Bowen_ratio':'B',
    'ZL': 'PBL Stability',
    'TKE': 'Specific Turb E',
    'rho_a':'rho_v'
}, inplace=True)
df['datetime']=pd.to_datetime(df['datetime'])
df.columns
# LWS_volt: Leaf sensor. Voltage output related to mass of standing water on a leaf
# G_HFP01_#: Soil heat flux measured by a single heat flux plate, GHFP01-(1-4)

In [ ]:
# change certain columns to datetime format
df['day']=df['datetime'].dt.date
df['hour']=df['datetime'].dt.hour
df['minute']=df['datetime'].dt.minute
df['hour']=df['hour'].astype(str).str.zfill(2)
df['minute']=df['minute'].astype(str).str.zfill(2)
df['hr:min']=df['hour'].astype(str) + ':' + df['minute'].astype(str)
df['date']=df['day'].astype(str)+ ' ' + df['hr:min'].astype(str)
df

In [ ]:
df_temp=df.loc[(df['datetime'].astype(str) >= '2023-06-06 18:00') & (df['datetime'].astype(str) < '2023-06-07 18:30')]
df_temp

In [ ]:
sites=df_temp['site ID'].unique()
sites

In [ ]:
# map lats and lons to sites
lat_vals=[43.78962, 40.63176, 43.31699, 44.89565, 41.9792, 
     42.41817, 43.25941, 42.02571, 42.65578, 40.73434,
     43.62218, 42.00168, 43.11700, 41.04018, 40.60401,
     42.65242, 42.77993, 43.48507]
lon_vals=[-76.11373, -73.95368, -78.74903, -73.46461, -74.5171,
    -79.3666, -77.37331, -76.25543, -76.98746, -73.81586,
    -75.87769, -73.88391, -73.57828, -72.46586, -74.1485,
    -73.97562, -78.20889, -73.42307]
lats = {
    'FLUX_BELL': lat_vals[0],
    'FLUX_BKLN': lat_vals[1],
    'FLUX_BURT': lat_vals[2],
    'FLUX_CHAZ': lat_vals[3],
    'FLUX_CLAR': lat_vals[4],
    'FLUX_FRED': lat_vals[5],
    'FLUX_ONTA': lat_vals[6],
    'FLUX_OWEG': lat_vals[7],
    'FLUX_PENN': lat_vals[8],
    'FLUX_QUEE': lat_vals[9],
    'FLUX_REDF': lat_vals[10],
    'FLUX_REDH': lat_vals[11],
    'FLUX_SCHU': lat_vals[12],
    'FLUX_SOUT': lat_vals[13],
    'FLUX_STAT': lat_vals[14],
    'FLUX_VOOR': lat_vals[15],
    'FLUX_WARS': lat_vals[16],
    'FLUX_WHIT': lat_vals[17]
}
lons = {
    'FLUX_BELL': lon_vals[0],
    'FLUX_BKLN': lon_vals[1],
    'FLUX_BURT': lon_vals[2],
    'FLUX_CHAZ': lon_vals[3],
    'FLUX_CLAR': lon_vals[4],
    'FLUX_FRED': lon_vals[5],
    'FLUX_ONTA': lon_vals[6],
    'FLUX_OWEG': lon_vals[7],
    'FLUX_PENN': lon_vals[8],
    'FLUX_QUEE': lon_vals[9],
    'FLUX_REDF': lon_vals[10],
    'FLUX_REDH': lon_vals[11],
    'FLUX_SCHU': lon_vals[12],
    'FLUX_SOUT': lon_vals[13],
    'FLUX_STAT': lon_vals[14],
    'FLUX_VOOR': lon_vals[15],
    'FLUX_WARS': lon_vals[16],
    'FLUX_WHIT': lon_vals[17]
}
df_temp=df_temp.copy()
df_temp['latitude'] = df_temp['site ID'].map(lats)
df_temp['longitude'] = df_temp['site ID'].map(lons)

In [ ]:
# PLOT ALL SITES
fig=plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='-')
ax.add_feature(cfeature.LAND, facecolor='black',alpha=0.3)
ax.add_feature(cfeature.LAKES, facecolor='lightblue', alpha=0.3)

cmap = cm.get_cmap('inferno', len(sites))

for i,site in enumerate(sites):
    lon=df_temp.loc[df_temp['site ID'] == site]['longitude'].unique()
    lat=df_temp.loc[df_temp['site ID']== site]['latitude'].unique()
    ax.plot(lon, lat, 's',markersize=10, color=cmap(i), label=f'{site[5::]}')
    ax.text(lon-0.15,lat-0.3,f'{site[5::]}', rotation=25, color=cmap(i),fontweight='bold', fontsize=12)

gl = ax.gridlines(draw_labels=True,linewidth=1, color='black', alpha=0.7, linestyle='--')
gl.top_labels = False
ax.set_extent([-80, -71, 40, 45.4])  
ax.legend(loc='upper right')
ax.set_title('Plot of NY Flux Sites', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('PART1_sites.png',dpi=300)

In [ ]:
# extract basic site name
suffix = [site.split("_")[1] for site in sites]
suffix

In [ ]:
# READ IN STANDARD SITE DATA - only extract data
df_standard = pd.read_csv('/home/jovyan/Desktop/Micrometeorology/DATA/NYSMesonet_standard_2023-06_06-07.csv')
df_standard = df_standard.loc[df_standard['station'].isin(suffix)]
# df_standard.station.unique()

# average for the hour
df_standard['time']=pd.to_datetime(df_standard['time'])
df_standard['day']=df_standard['time'].dt.date
df_standard['hour']=df_standard['time'].dt.hour
df_standard['time']=df_standard['time'].dt.strftime('%Y-%m-%d %H:%M')


df_standard=df_standard.groupby(['station','day', 'hour'],as_index=False).mean(numeric_only=True)
df_standard['datetime']=pd.to_datetime(df_standard['day'].astype(str) + ' ' + df_standard['hour'].astype(str).str.zfill(2) +':00')
df_standard=df_standard.loc[(df_standard['datetime'] >= '2023-06-06 18:00') & (df_standard['datetime'] <= '2023-06-07 18:00')]
df_standard['datetime'] = pd.to_datetime(df_standard['datetime'])
df_standard.station.unique()

# SURFACE CALCULATIONS: CALCULATE HG
Equation is $H_G = H_{6cm} + C_s \Delta z_A \frac{δT_{SFC}}{δt} + C_s \Delta z_B \frac{δT_{5cm}}{δt}$ </p>

In [ ]:
# read in soil data
df_soil = pd.read_csv('/home/jovyan/Desktop/Micrometeorology/DATA/soil_2023-06_06-07.csv')
# list standard sites
standard_sites=df_soil['station'].unique()
standard_sites

# list flux sites
flux_sites = ['FLUX_BELL','FLUX_BKLN','FLUX_BURT','FLUX_CHAZ','FLUX_CLAR','FLUX_FRED','FLUX_ONTA','FLUX_OWEG','FLUX_PENN','FLUX_QUEE','FLUX_REDF','FLUX_REDH','FLUX_SCHU','FLUX_SOUT','FLUX_STAT','FLUX_VOOR','FLUX_WARS','FLUX_WHIT']
suffix = [site.split("_")[1] for site in flux_sites]

# if station is in suffix, then keep
df_soil=df_soil.loc[df_soil['station'].isin(suffix)]
df_soil['datetime']=pd.to_datetime(df_soil['time'])
df_soil['datetime']=df_soil['datetime'].dt.tz_localize(None) # remove +00..
df_soil['hour'], df_soil['minute']=df_soil['datetime'].dt.hour, df_soil['datetime'].dt.minute
df_soil['day'] = df_soil['datetime'].dt.day
df_soil=df_soil.loc[(df_soil['datetime'] >= '2023-06-06 18:00') & (df_soil['datetime'] <= '2023-06-07 18:30')]
df_soil=df_soil.loc[(df_soil['minute']==0) | (df_soil['minute']==30)]

# read in flux data for RL (LW_OUT)
df_RL = pd.read_csv('/home/jovyan/Desktop/Micrometeorology/DATA/new_flux_pblh.csv')

df_RL['stid']=df_RL['stid'].str.split("_").str[1]
df_RL = df_RL.rename(columns={'stid':'station'})
df_RL['datetime']=pd.to_datetime(df_RL['datetime'])
df_RL['day'], df_RL['hour'], df_RL['minute'] = df_RL['datetime'].dt.day, df_RL['datetime'].dt.hour, df_RL['datetime'].dt.minute
df_RL=df_RL.loc[(df_RL['datetime'] >= '2023-06-06 18:00') & (df_RL['datetime'] <= '2023-06-07 18:30')]
df_RL['G_6cm'] = df_RL[['G_HFP01_1','G_HFP01_2', 'G_HFP01_3','G_HFP01_4']].astype(float).mean(axis=1)

# merge on station and datetime
df_HG = pd.merge(
    df_soil,
    df_RL[['station','datetime','LW_OUT','G_6cm']],  # only keep relevant columns
    on=['station','datetime'],
    how='left'  # keep all soil rows, add RL fluxes where available
)

# # make a new df for simplicity, only picking a few values out
HG = {
    'site ID': df_HG['station'].values,
    'datetime': df_HG['datetime'].values,
    'day': df_HG['day'].values,
    'minute': df_HG['minute'].values,
    'hour': df_HG['hour'].values,
    'soil_moisture_5cm': df_HG['soil_moisture_05cm [m^3/m^3]'].values,
    'soil_temp_5cm': df_HG['soil_temp_05cm [degC]'].values,
    'LW_OUT': df_HG['LW_OUT'].values,
    'G_6cm': df_HG['G_6cm'].values
}
HG_df = pd.DataFrame(HG)
# df_HG.to_csv('HG.csv',index=False)
# print(df_RL.loc[df_RL['stid']=='CLAR']['G_6cm'])
HG_df

In [ ]:
# declare constant values
    # Cs = (phi_m)(rho_m)(c_m) + (theta)(rho_w)(c_w) + (phi_0)(rho_0)(c_0)
phi_m=0.45 # m3/m3
rho_m=2650 # kg/m3
c_m=870 # J/(kg K)
Cs_1 = phi_m*rho_m*c_m

# theta: pulled from 00; find site's 00 value of soil_moisture_05 cm [m^3/m^3]
rho_w=999.7 # kg/m3
c_w=4192 # J/(kg K)

phi_0=0.05 # m3/3
rho_0=1300 # kg/3
c_0=1920 # J/(kg K)
Cs_3 = phi_0*rho_0*c_0
sec=3600

# R_L_00: pull from flux where minute == 0
# R_L_30: pull from flux where minute == 30
# soil_temp_00: pull from soil where minute == 0
# soil_temp_30: pull from soil where minute == 30
zA, zB = 0.01, 0.05 # m = 1 cm
emissivity = 0.98
stefan = (5.67e8) # W/(m2 K4)
stefan

In [ ]:
# create empty dictionary
HG_dict = {
    'site ID': [],
    'datetime': [],
    'HG': []
}

# for each day (6 and 7), for hour in 24 hours, for each site, calculate HG
days = HG_df['day'].unique()
sites = HG_df['site ID'].unique()

for day in days:
    # set valid hours depending on the day
    if day == 6:
        hours = list(range(18, 24))  # 18,19,20,21,22,23
    elif day == 7:
        hours = list(range(0, 19))   # 0 through 18
    else:
        continue  # skip other days if any

    for hour in hours:
        for site in sites:    
            # select the data
            df_hg = HG_df.loc[(HG_df['site ID'] == site) & (HG_df['day'] == day) & (HG_df['hour'] == hour)]
            # print(df_temp.iloc[0]) # corresponds to the hour (00)
            # print(df_temp.iloc[1]) # corresponds to half past the hour (30)
            if df_hg.empty:
                print(f"No data for {site} day {day} hour {hour}")
                continue  
                
            if len(df_hg) < 2:
                print(f"Skipping {site} day {day} hour {hour} — no value for minute 30")
                continue  # need both 0-min and 30-min rows

            # divide up into parts
            p1 = df_hg['G_6cm'].iloc[0]

            theta = df_hg['soil_moisture_5cm'].iloc[0]
            p2 = Cs_1 + ((theta)*(rho_w)*(c_w)) + Cs_3 # Cs

            p3 = zA

        
            RL_30 = df_hg['LW_OUT'].iloc[1]
            RL_00 = df_hg['LW_OUT'].iloc[0]
            p_4_30 = ((RL_30)/(emissivity*stefan))**(0.25)
            p_4_00 = ((RL_00)/(emissivity*stefan))**(0.25)
            p4 = (p_4_30 - p_4_00)/sec

            p5 = zB

            p_6_30 = df_hg['soil_temp_5cm'].iloc[1]+273.15 # convert from C to K
            p_6_00 = df_hg['soil_temp_5cm'].iloc[0]+273.15 # convert from C to K
            p6 = (p_6_30 - p_6_00)/sec
            
            Hg = p1 + p2*p3*p4 + p2*p5*p6

            HG_dict['site ID'].append(site)
            HG_dict['datetime'].append(df_hg['datetime'].iloc[0])  # use 0-min row
            HG_dict['HG'].append(Hg)

# convert dictionary to DataFrame
df_HG = pd.DataFrame(HG_dict)

# map lats and lons to sites
lat_vals=[43.78962, 40.63176, 43.31699, 44.89565, 41.9792, 
     42.41817, 43.25941, 42.02571, 42.65578, 40.73434,
     43.62218, 42.00168, 43.11700, 41.04018, 40.60401,
     42.65242, 42.77993, 43.48507]
lon_vals=[-76.11373, -73.95368, -78.74903, -73.46461, -74.5171,
    -79.3666, -77.37331, -76.25543, -76.98746, -73.81586,
    -75.87769, -73.88391, -73.57828, -72.46586, -74.1485,
    -73.97562, -78.20889, -73.42307]
lats = {
    'BELL': lat_vals[0],
    'BKLN': lat_vals[1],
    'BURT': lat_vals[2],
    'CHAZ': lat_vals[3],
    'CLAR': lat_vals[4],
    'FRED': lat_vals[5],
    'ONTA': lat_vals[6],
    'OWEG': lat_vals[7],
    'PENN': lat_vals[8],
    'QUEE': lat_vals[9],
    'REDF': lat_vals[10],
    'REDH': lat_vals[11],
    'SCHU': lat_vals[12],
    'SOUT': lat_vals[13],
    'STAT': lat_vals[14],
    'VOOR': lat_vals[15],
    'WARS': lat_vals[16],
    'WHIT': lat_vals[17]
}
lons = {
    'BELL': lon_vals[0],
    'BKLN': lon_vals[1],
    'BURT': lon_vals[2],
    'CHAZ': lon_vals[3],
    'CLAR': lon_vals[4],
    'FRED': lon_vals[5],
    'ONTA': lon_vals[6],
    'OWEG': lon_vals[7],
    'PENN': lon_vals[8],
    'QUEE': lon_vals[9],
    'REDF': lon_vals[10],
    'REDH': lon_vals[11],
    'SCHU': lon_vals[12],
    'SOUT': lon_vals[13],
    'STAT': lon_vals[14],
    'VOOR': lon_vals[15],
    'WARS': lon_vals[16],
    'WHIT': lon_vals[17]
}
df_HG['latitude'] = df_HG['site ID'].map(lats)
df_HG['longitude'] = df_HG['site ID'].map(lons)

df_HG['site ID'] = 'FLUX_' + df_HG['site ID']
df_HG

In [ ]:
# reframe above code:
# drop where min = 30
df_temp=df_temp.loc[df_temp['minute'] != '30'].copy()

In [ ]:
# add HG calculated values back to df_temp
df_temp = df_temp.merge(
    df_HG[['site ID', 'datetime', 'HG']], 
    on=['site ID', 'datetime'], 
    how='left'  # keeps all rows in df_main
)
df_temp

# SURFACE CALCULATIONS: CALCULATE RESIDUAL TERM

In [ ]:
df_temp['residual'] = df_temp['Rn'].astype(float) - ( df_temp['H'].astype(float) + df_temp['HL'].astype(float) + df_temp['HG'].astype(float) )
df_temp

In [ ]:
# reframe above code:
# drop where min = 30
df_temp=df_temp.loc[df_temp['minute'] != '30'].copy()
# get unique sitenames
sites=df['site ID'].unique()

In [ ]:
# specifications for loop
savepath='/home/jovyan/Desktop/Micrometeorology/deliv4_P1/' # need to specify folder to save to in the loop
vois = ['SHTFL', 'LHTFL', 'USWRF', 'DSWRF', 'ULWRF', 'DLWRF', 'GFLUX',\
        'TMP', 'DPT', 'UGRD', 'VGRD','WIND', 'VIS'] # used for HRRR data 

units = ['W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$',\
         '$^{o}$C','$^{o}$C','m s$^{-1}$','m s$^{-1}$','m s$^{-1}$', 'mi']

titles=['Sensible Heat Flux','Latent Heat Flux', 'Outgoing Shortwave Radiation', 'Incoming Shortwave Radiation', 'Outgoing Longwave Radiation', 'Incoming Longwave Radiation', 'Ground Heat Flux', \
        'Temperature', 'Dewpoint Temperature', 'U-Component of Wind', 'V-Component of Wind', 'Wind Speed', 'Visibility']

voi_df2=['H','HL','SW_OUT','SW_IN','LW_OUT','LW_IN', 'HG',\
        'temp_2m [degC]', 'dewpoint [degC]', 'wind_direction_merge [degrees]', 'avg_wind_speed_merge [m/s]', 'VOID'] # corresponds to NYS Mesonet data. Void = no vis data

# plot info
lon_min, lon_max = -80, -72
lat_min, lat_max = 40.4, 45.3
bbox = (lon_min, lat_min, lon_max, lat_max) 
# time info
times=df_temp['datetime'].values.copy() 
    # if you need an hourly forecast, then you'll need to initialize at a time and then increase the forecast lead time to fill gaps. thus,
#forecast_hrs = [24] # 24 as a baseline, then do 25, 26, 27, 28, 29
# set 1 initialization time and forecast 24 hours from there. so start this as an initialized time of 2023-05-18. forecast 24 hours until you reach hour 48, 18Z on 7th
mesonet_hours=df_temp['datetime'].unique()
standard_site_hours=df_standard['datetime'].unique()
for_init_time='2023-06-05 18:00'
forecast_hrs = list(range(24,49))

In [ ]:
# specify voi
num=0 # used to access both voi_df2 indexing (0='H') and for titles indexing (0='Sensible ...') so change based on voi
v=vois[num]
unit=units[num]

# set cmap based on variable
cmaps=['Spectral', 'winter', 'spring', 'summer', 'cool','autumn', 'pink', \
      'copper','Greens', 'viridis','viridis', 'Purples', 'RdPu']
cmap=cmaps[num]

# set values for vmin and vmax depending on the variable
vmins=[-150,-5,-30, 0, 150,230, -40,\
       5, -10, 0, 0, 0,0]
vmaxs=[700,310,200, 1300, 450, 420, 40,\
       30, 15, 10, 5, 10,30]
vmin,vmax=vmins[num],vmaxs[num]


for i in range(0,len(forecast_hrs)): 
    t=times[i]
    # pull the hrrr data
    hrrr = Herbie(
        for_init_time, # model initialization time
        model='hrrr', # model
        fxx = forecast_hrs[i] # forecast lead time (hrs)
             )
    hrrr.download() # download hrrr data
    
    if v=='TMP' or v=='DPT':
        inventory=hrrr.inventory(v) # search for voi # SHTFL
        hrrr_data=hrrr.xarray(inventory['search_this'].iloc[-1])
        tem=hrrr_data[list(hrrr_data.data_vars)[0]].values-273.15
        hrrr_data[list(hrrr_data.data_vars)[0]].values=tem
    elif v=='UGRD' or v=='VGRD':
        inventory=hrrr.inventory(v) # search for voi # SHTFL
        hrrr_data=hrrr.xarray(inventory['search_this'].iloc[-1])
    elif v=='VIS':
        inventory=hrrr.inventory(v) # search for voi # SHTFL
        hrrr_data=hrrr.xarray(inventory['search_this'].iloc[0])*0.000621371
    else:
        inventory=hrrr.inventory(v) # search for voi # SHTFL
        hrrr_data=hrrr.xarray(inventory['search_this'].iloc[0])
    
    
   #  # plot the hrrr data
    fig=plt.figure(figsize=(10,10)) # create figure
    ax = plt.axes(projection=ccrs.PlateCarree())
    mesh = ax.pcolormesh(hrrr_data['longitude'],hrrr_data['latitude'],hrrr_data[list(hrrr_data.data_vars)[0]], cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree(),zorder=0) # background
    # set the colorbar
    cbar = fig.colorbar(mesh, ax=ax, orientation='horizontal', shrink = 1, pad=0.04)  
    cbar.set_label(f'{titles[num]} ({unit})', fontsize = 16, fontweight = 'bold', labelpad=8)
    cbar.ax.tick_params(labelsize=14)
    t=mesonet_hours[i]
    print(t)

    if v=='TMP' or v=='DPT':
        # pull the corresponding mesonet data
        df_s2 = df_standard[df_standard['datetime'] == t]
        # plot the mesonet data
        cs=ax.scatter(df_s2['longitude [degrees_east]'],df_s2['latitude [degrees_north]'],c=df_s2[voi_df2[num]],marker='s',s=100,cmap=cmap,vmin=vmin,vmax=vmax,
                edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
        ax.legend(loc=3,fontsize=12)
        transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
    
    elif v=='UGRD' or v=='VGRD':
        # pull the corresponding mesonet data
        df_s2 = df_standard[df_standard['datetime'] == t]
        # plot the mesonet data
        cs=ax.scatter(df_s2['longitude [degrees_east]'],df_s2['latitude [degrees_north]'],c=df_s2[voi_df2[9]],marker='s',s=100,cmap=cmap,vmin=vmin,vmax=vmax,
                edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
        ax.legend(loc=3,fontsize=12)
        transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
    elif v=='WIND':
        # pull the corresponding mesonet data
        df_s2 = df_standard[df_standard['datetime'] == t]
        # plot the mesonet data
        cs=ax.scatter(df_s2['longitude [degrees_east]'],df_s2['latitude [degrees_north]'],c=df_s2[voi_df2[10]],marker='s',s=100,cmap=cmap,vmin=vmin,vmax=vmax,
                edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
        ax.legend(loc=3,fontsize=12)
        transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
    elif v=='VIS':
        print('ahahaha') # filler block since there is no direct data measure of visibility from NYS Mesonet
    else:
        # pull the corresponding mesonet data
        df_2 = df_temp[df_temp['datetime'] == t]
        df_2=df_2.dropna(subset=['H','HL','HG','Rn','SW_IN','SW_OUT','LW_IN','LW_OUT'])
        
        # plot the mesonet data
        cs=ax.scatter(df_2['longitude'],df_2['latitude'],c=df_2.loc[:, voi_df2[num]],marker='s',s=100,cmap=cmap,vmin=vmin,vmax=vmax,
                edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
        ax.legend(loc=3,fontsize=12)
        transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
        
    # add map features
    ax.set_xlim([bbox[0], bbox[2]])
    ax.set_ylim([bbox[1], bbox[3]])
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, edgecolor = 'black')
    ax.add_feature(cfeature.STATES, edgecolor = 'black')
    ax.coastlines()
    # edit the gridlines
    gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, color = 'k', alpha=0.7, linewidth=0.5, linestyle='--')
    gl.right_labels = False  
    gl.top_labels = False   
    gl.xlabel_style = {'size':14}
    gl.ylabel_style = {'size':14}

    # set the title and save the figure
    ax.set_title(f'HRRR F{forecast_hrs[i]} Model Field versus Observed NYS Mesonet\n {titles[num]} Values for {t}Z', fontsize=18, fontweight = 'bold', pad = 10)
    plt.tight_layout()
    plt.savefig(f'{savepath}{v}/{v}_{forecast_hrs[i]}.png',dpi=300)

## NET RADIATION: $R_N = H+H_L+H_G$

In [ ]:
# specify colormap and colorbar range for RN
cmap='cividis'
vmin,vmax = -100,550
# variables
vois = ['SHTFL', 'LHTFL', 'USWRF', 'DSWRF', 'ULWRF', 'DLWRF', 'HPBL', 'GFLUX'] #'TMP', 'DPT', ] # used for HRRR data

for i in range(0,len(forecast_hrs)):    
    # pull the hrrr data
    hrrr = Herbie(
        for_init_time, # model initialization time
        model='hrrr', # model
        fxx = forecast_hrs[i] # forecast lead time (hrs)
             )
    hrrr.download() # download hrrr data
    # SENSIBLE HEAT - H
    inventory_H=hrrr.inventory(vois[0]) # search for voi # SHTFL
    hrrr_data_H=hrrr.xarray(inventory_H['search_this'].iloc[0])

    # LATENT HEAT - HL
    inventory_HL=hrrr.inventory(vois[1]) # search for voi # SHTFL
    hrrr_data_HL=hrrr.xarray(inventory_HL['search_this'].iloc[0])

    # GROUND HEAT - HG
    inventory_HG=hrrr.inventory(vois[7]) # search for voi # SHTFL
    hrrr_data_HG=hrrr.xarray(inventory_HG['search_this'].iloc[0])

    # CALCULATE RN
    hrrr_data = hrrr_data_HG[list(hrrr_data_HG.data_vars)[0]] + hrrr_data_HL[list(hrrr_data_HL.data_vars)[0]] + hrrr_data_H[list(hrrr_data_H.data_vars)[0]]
   
    
    # plot the hrrr data
    fig=plt.figure(figsize=(10,10)) # create figure
    ax = plt.axes(projection=ccrs.PlateCarree())
    mesh = ax.pcolormesh(hrrr_data['longitude'],hrrr_data['latitude'],hrrr_data, cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree(),zorder=0) # background
    # set the colorbar
    cbar = fig.colorbar(mesh, ax=ax, orientation='horizontal', shrink = 1, pad=0.04)  
    cbar.set_label(f'Net Radiation ({unit})', fontsize = 16, fontweight = 'bold', labelpad=8)
    cbar.ax.tick_params(labelsize=14)
    
    # pull the corresponding mesonet data
    t=mesonet_hours[i]
    df_2 = df_temp[df_temp['datetime'] == t]
    df_2=df_2.dropna(subset=['H','HL','HG','Rn','SW_IN','SW_OUT','LW_IN','LW_OUT'])
    
    # plot the mesonet data
    cs=ax.scatter(df_2['longitude'],df_2['latitude'],c=df_2['Rn'],marker='s',s=100,cmap=cmap,vmin=vmin,vmax=vmax,
            edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
    ax.legend(loc=3,fontsize=12)
    transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
    
    
    # add map features
    ax.set_xlim([bbox[0], bbox[2]])
    ax.set_ylim([bbox[1], bbox[3]])
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, edgecolor = 'black')
    ax.add_feature(cfeature.STATES, edgecolor = 'black')
    ax.coastlines()
    # edit the gridlines
    gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, color = 'k', alpha=0.7, linewidth=0.5, linestyle='--')
    gl.right_labels = False  
    gl.top_labels = False   
    gl.xlabel_style = {'size':14}
    gl.ylabel_style = {'size':14}


    # set the title and save the figure
    ax.set_title(f'HRRR F{forecast_hrs[i]} Model Field versus Observed NYS Mesonet\n Net Radiation Values for {t}Z', fontsize=18, fontweight = 'bold', pad = 10)
    plt.tight_layout()
    plt.savefig(f'{savepath}RN/RN_{forecast_hrs[i]}.png',dpi=300)

## RESIDUAL TERM: $ Residual = R_N - (H+H_L+H_G)$

In [ ]:
# variables
vois = ['SHTFL', 'LHTFL', 'USWRF', 'DSWRF', 'ULWRF', 'DLWRF', 'HPBL', 'GFLUX'] #'TMP', 'DPT', ] # used for HRRR data
# colorbar range
vmin, vmax = -100,100

for i in range(0,len(forecast_hrs)):    
    # pull the hrrr data
    hrrr = Herbie(
        for_init_time, # model initialization time
        model='hrrr', # model
        fxx = forecast_hrs[i] # forecast lead time (hrs)
             )
    hrrr.download() # download hrrr data
    # SENSIBLE HEAT - H
    inventory_H=hrrr.inventory(vois[0]) # search for voi # SHTFL
    hrrr_data_H=hrrr.xarray(inventory_H['search_this'].iloc[0])

    # LATENT HEAT - HL
    inventory_HL=hrrr.inventory(vois[1]) # search for voi # SHTFL
    hrrr_data_HL=hrrr.xarray(inventory_HL['search_this'].iloc[0])

    # GROUND HEAT - HG
    inventory_HG=hrrr.inventory(vois[7]) # search for voi # SHTFL
    hrrr_data_HG=hrrr.xarray(inventory_HG['search_this'].iloc[0])

    # CALCULATE RN
    hrrr_data_RN = hrrr_data_HG[list(hrrr_data_HG.data_vars)[0]] + hrrr_data_HL[list(hrrr_data_HL.data_vars)[0]] + hrrr_data_H[list(hrrr_data_H.data_vars)[0]]

    # CALCULATE RESIDUAL TERM
    hrrr_data = hrrr_data_RN - (hrrr_data_HG[list(hrrr_data_HG.data_vars)[0]] + hrrr_data_HL[list(hrrr_data_HL.data_vars)[0]] + hrrr_data_H[list(hrrr_data_H.data_vars)[0]])
    
    # plot the hrrr data
    fig=plt.figure(figsize=(10,10)) # create figure
    ax = plt.axes(projection=ccrs.PlateCarree())
    mesh = ax.pcolormesh(hrrr_data['longitude'],hrrr_data['latitude'],hrrr_data, cmap='coolwarm',vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree(),zorder=0) # background
    # set the colorbar
    cbar = fig.colorbar(mesh, ax=ax, orientation='horizontal', shrink = 1, pad=0.04)  
    cbar.set_label('Residual Term (W m$^{-2}$)', fontsize = 16, fontweight = 'bold', labelpad=8)
    cbar.ax.tick_params(labelsize=14)

    # pull the corresponding mesonet data
    t=mesonet_hours[i]
    df_2 = df_temp[df_temp['datetime'] == t]
    df_2=df_2.dropna(subset=['H','HL','HG','Rn','SW_IN','SW_OUT','LW_IN','LW_OUT'])
    
    # plot the mesonet data
    cs=ax.scatter(df_2['longitude'],df_2['latitude'],c=df_2['residual'],marker='s',s=100,cmap='coolwarm',vmin=-100,vmax=100,
            edgecolor='black',linewidth=2,transform=ccrs.PlateCarree(),label='NYS Mesonet Flux Sites',zorder=4)    
    ax.legend(loc=3,fontsize=12)
    transform = ccrs.PlateCarree()._as_mpl_transform(ax) # set transform for annotations
    
    # add map features
    ax.set_xlim([bbox[0], bbox[2]])
    ax.set_ylim([bbox[1], bbox[3]])
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, edgecolor = 'black')
    ax.add_feature(cfeature.STATES, edgecolor = 'black')
    ax.coastlines()
    # edit the gridlines
    gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, color = 'k', alpha=0.7, linewidth=0.5, linestyle='--')
    gl.right_labels = False  
    gl.top_labels = False   
    gl.xlabel_style = {'size':14}
    gl.ylabel_style = {'size':14}

    # set the title and save the figure
    ax.set_title(f'HRRR F{forecast_hrs[i]} Model Field versus Observed NYS Mesonet\n Residual Term Values for {t}Z', fontsize=18, fontweight = 'bold', pad = 10)
    plt.tight_layout()
    plt.savefig(f'{savepath}RESIDUAL/RESIDUAL_{forecast_hrs[i]}.png',dpi=300)

# HRRR and NYS Mesonet Plots - Line Plots
Examining 5 sites: Main site of interest is OWEG site. 4 other sites are PENN, CLAR, ONTA, and WARS

In [ ]:
# find where time =! 30 and data for specified sites
df_temp=df_temp.loc[df_temp['minute'] != '30'].copy()
df_2 = df_temp.loc[df_temp['site ID'].isin(['FLUX_OWEG', 'FLUX_PENN', 'FLUX_CLAR', 'FLUX_ONTA', 'FLUX_WARS'])] # for flux data
df_temp2 = df_standard.loc[df_standard['station'].isin(['OWEG', 'PENN', 'CLAR', 'ONTA', 'WARS'])] # for standard site data

In [ ]:
# specifications for loop
savepath='/home/jovyan/Desktop/Micrometeorology/deliv4_P2/' # need to specify folder to save to in the loop
vois = ['SHTFL', 'LHTFL', 'USWRF', 'DSWRF', 'ULWRF', 'DLWRF', 'GFLUX',\
        'TMP', 'DPT', 'UGRD', 'VGRD','WIND'] # used for HRRR data 
# for T, Td, U/VGRD, SPD, you need to specify level, so be careful
units = ['W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$','W m$^{-2}$', 'W m$^{-2}$',\
         '$^{o}$C','$^{o}$C','m s$^{-1}$','m s$^{-1}$','m s$^{-1}$']

titles=['Sensible Heat Flux','Latent Heat Flux', 'Outgoing Shortwave Radiation', 'Incoming Shortwave Radiation', 'Outgoing Longwave Radiation', 'Incoming Longwave Radiation', 'Ground Heat Flux',\
        'Temperature', 'Dewpoint Temperature', 'U-Component of Wind', 'V-Component of Wind', 'Wind Speed']

voi_df2=['H','HL','SW_OUT','SW_IN','LW_OUT','LW_IN', 'HG',\
        'temp_2m [degC]', 'dewpoint [degC]', 'wind_direction_merge [degrees]', 'avg_wind_speed_merge [m/s]'] # corresponds to NYS Mesonet data

# get corresponding NYS Mesonet data
times=df_temp['datetime'].values.copy() 

# set 1 initialization time and forecast 24 hours from there. so start this as an initialized time of 2023-05-18. forecast 24 hours until you reach hour 48, 18Z on 7th
mesonet_hours=df_2['datetime'].unique()
for_init_time='2023-06-05 18:00'
forecast_hrs = list(range(24,49))

In [ ]:
# get sitenames and forecast hours
sites = df_2['site ID'].unique()
forecast_hours = forecast_hrs
# create an array for model data
model_data = []

for fxx in forecast_hours:
    hrrr = Herbie(for_init_time, model='hrrr', fxx=fxx)
    hrrr.download()
    
    for site in sites:
        target_lat = df_temp.loc[df_temp['site ID'] == site, 'latitude'].values[0]
        target_lon = df_temp.loc[df_temp['site ID'] == site, 'longitude'].values[0]
        if target_lon < 0:
            target_lon = target_lon % 360
        
        row = {'site': site, 'forecast_hour': fxx}
        
        # Extract each variable
        data_arrays = {}
        for v in vois:
            inventory = hrrr.inventory(v)
            if len(inventory) == 0:
                data_arrays[v] = np.nan
                continue  # skip missing VOIs
            
            # Handle specific VOIs differently
            if v in ['TMP', 'DPT']:
                hrrr_data = hrrr.xarray(inventory['search_this'].iloc[-1])
                # Convert from K to °C
                var_name = list(hrrr_data.data_vars)[0]
                da = hrrr_data[var_name] - 273.15
            else:
                hrrr_data = hrrr.xarray(inventory['search_this'].iloc[0])
                var_name = list(hrrr_data.data_vars)[0]
                da = hrrr_data[var_name]
            
            # Find nearest grid point
            distances = (hrrr_data['latitude'] - target_lat)**2 + (hrrr_data['longitude'] - target_lon)**2
            y, x = np.unravel_index(np.argmin(distances.values), distances.shape)
            data_arrays[v] = da.isel(y=y, x=x).item()
        
        # Add all VOIs to the row
        for v in vois:
            row[v] = data_arrays.get(v, np.nan)
        
        # Compute RN = SHTFL + LHTFL + GRDFLX
        row['SHTFL'] = data_arrays.get('SHTFL', np.nan)
        row['LHTFL'] = data_arrays.get('LHTFL', np.nan)
        row['GFLUX'] = data_arrays.get('GFLUX', np.nan)
        row['USWRF'] = data_arrays.get('USWRF', np.nan)
        row['DSWRF'] = data_arrays.get('DSWRF', np.nan)
        row['ULWRF'] = data_arrays.get('ULWRF', np.nan)
        row['DLWRF'] = data_arrays.get('DLWRF', np.nan)
        row['HPBL'] = data_arrays.get('HPBL', np.nan)
        row['RN_SFCBUDGET'] = sum([data_arrays.get(v, 0) for v in ['SHTFL', 'LHTFL', 'GFLUX'] if not np.isnan(data_arrays.get(v, np.nan))])
        row['RN_RAD'] = sum([data_arrays.get(v, 0) for v in ['USWRF', 'DSWRF', 'ULWRF', 'DLWRF'] if not np.isnan(data_arrays.get(v, np.nan))])
        row['WIND'] = data_arrays.get('WIND', np.nan)
        row['U'] = data_arrays.get('UGRD', np.nan)
        row['V'] = data_arrays.get('VGRD', np.nan)
        row['TMP'] = data_arrays.get('TMP', np.nan)
        row['DPT'] = data_arrays.get('DPT', np.nan)

        model_data.append(row)

# Convert to DataFrame
model_df = pd.DataFrame(model_data)

In [ ]:
model_df

In [ ]:
# add station name column and datetime column (strings)
df_temp2['station_flux'] = 'FLUX_' + df_temp2['station'].astype(str)
df_temp2['datetime'] = df_temp2['day'].astype(str) + ' ' + df_temp2['hour'].astype(str) + ':00Z'
df_temp2

In [ ]:
# PLOT THE DATA
name = 'temp' # choose either wind, temp, sfc, or rad

# TMP, DPT, WIND
for i, site in enumerate(sites):
    print(site)
    df_obs = df_temp.loc[df_temp['site ID'] == site]
    df_obs_stand = df_temp2.loc[df_temp2['station_flux'] == site]
    df_mod = model_df.loc[model_df['site']==site]
    
    # TIME
    time = df_obs_stand['datetime'].copy()
    
    # MODEL DATA
    h_mod = df_mod['SHTFL'].copy()
    hl_mod = df_mod['LHTFL'].copy()
    hg_mod = df_mod['GFLUX'].copy()
    rn_mod = h_mod + hl_mod
    usw_mod = df_mod['USWRF'].copy()
    dsw_mod = df_mod['DSWRF'].copy()
    ulw_mod = df_mod['ULWRF'].copy()
    dlw_mod = df_mod['DLWRF'].copy()
    rn_mod_rad = usw_mod + dsw_mod + ulw_mod + dlw_mod
    tmp_mod = df_mod['TMP'].copy()
    dpt_mod = df_mod['DPT'].copy()
    spd_mod = df_mod['WIND'].copy()

    # OBSERVED DATA
    h_obs = df_obs['H'].copy()
    hl_obs = df_obs['HL'].copy()
    hg_obs = df_obs['HG'].copy()
    rn_obs = h_obs + hl_obs
    usw_obs = df_obs['SW_OUT'].copy()
    dsw_obs = df_obs['SW_IN'].copy()
    ulw_obs = df_obs['LW_OUT'].copy()
    dlw_obs = df_obs['LW_IN'].copy()
    rn_obs_rad = usw_obs + dsw_obs + ulw_obs + dlw_obs
    tmp_obs = df_obs_stand['temp_2m [degC]'].copy()
    dpt_obs = df_obs_stand['dewpoint [degC]'].copy()
    spd_obs = df_obs_stand['avg_wind_speed_merge [m/s]'].copy()


    ### PLOTTING ###
    if name == 'wind':
        # Plot
        fig, ax = plt.subplots(figsize=(15,5))
        ax.plot(time, spd_obs, color='darkslategrey', label='Observed Wind Speed')
        ax.plot(time, spd_mod, color='darkslategrey', linestyle='--', label='Model Wind Speed')
    
        ax.set_xticks(time[::3])
        ax.tick_params(axis='x', labelsize=12, rotation=25)
        ax.tick_params(axis='y', labelsize=12)
        
        legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
        legend.get_frame().set_edgecolor('black')
        legend.get_frame().set_linewidth(0.8)
        legend.get_frame().set_facecolor('snow')
        
        ax.set_ylabel('m $^{-1}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
        ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
        ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet \n Wind Speed \n{time.iloc[0]}Z to {time.iloc[-1]}Z at {site}', fontsize=18, fontweight='bold')
        
        ax.grid(True, alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'{savepath}WIND_{site}.png', dpi=300)
    elif name=='temp':
        # Plot
        fig, ax = plt.subplots(figsize=(15,5))
        ax.plot(time, tmp_obs, color='firebrick', label='Observed TMP')
        ax.plot(time, tmp_mod, color='firebrick', linestyle='--', label='Model TMP')
        ax.plot(time, dpt_obs, color='green', label='Observed DPT')
        ax.plot(time, dpt_mod, color='green', linestyle='--', label='Model DPT')
        
        ax.set_xticks(time[::3])
        ax.tick_params(axis='x', labelsize=12, rotation=25)
        ax.tick_params(axis='y', labelsize=12)
        
        legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
        legend.get_frame().set_edgecolor('black')
        legend.get_frame().set_linewidth(0.8)
        legend.get_frame().set_facecolor('snow')
        
        ax.set_ylabel('$^{o}$C', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
        ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
        ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet \n 2-meter Temperature & Dewpoint Temperatures \n{time.iloc[0]}Z to {time.iloc[-1]}Z at {site}', fontsize=18, fontweight='bold')
        
        ax.grid(True, alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'{savepath}TMPDPT_{site}.png', dpi=300)
    elif name =='sfc':
        # Plot
        fig, ax = plt.subplots(figsize=(15,5))
        ax.plot(time, h_obs, color='red', label='Observed H')
        ax.plot(time, hl_obs, color='navy', label='Observed HL')
        ax.plot(time, hg_obs, color='limegreen', label='Observed HG')
        ax.plot(time, rn_obs, color='plum', label='Observed RN')
        ax.plot(time, h_mod, color='red', linestyle='--', label='Model H')
        ax.plot(time, hl_mod, color='navy', linestyle='--', label='Model HL')
        ax.plot(time, hg_mod, color='limegreen', linestyle='--', label='Model HG')
        ax.plot(time, rn_mod, color='plum', linestyle='--', label='Model RN')
        
        ax.set_xticks(time[::3])
        ax.tick_params(axis='x', labelsize=12, rotation=25)
        ax.tick_params(axis='y', labelsize=12)
        
        legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
        legend.get_frame().set_edgecolor('black')
        legend.get_frame().set_linewidth(0.8)
        legend.get_frame().set_facecolor('snow')
        
        ax.set_ylabel('W m$^{-2}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
        ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
        ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet Surface Energy Budget\n{time.iloc[0]}Z to {time.iloc[-1]}Z at {site}', fontsize=18, fontweight='bold')
        
        ax.grid(True, alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'{savepath}SFCBUD_{site}.png', dpi=300)
    elif name=='rad':
        # Plot
        fig, ax = plt.subplots(figsize=(15,5))
        ax.plot(time, usw_obs, color='forestgreen', label='Observed USWRF')
        ax.plot(time, usw_mod, color='forestgreen', linestyle='--', label='Model USWRF')
        ax.plot(time, dsw_obs, color='darkorange', label='Observed DSWRF')
        ax.plot(time, dsw_mod, color='darkorange', linestyle='--',label='Model DSWRF')
        
        ax.plot(time, ulw_obs, color='dodgerblue', label='Observed ULWRF')
        ax.plot(time, ulw_mod, color='dodgerblue', linestyle='--', label='Model ULWRF')
        ax.plot(time, dlw_obs, color='mediumvioletred', label='Observed DLWRF')
        ax.plot(time, dlw_mod, color='mediumvioletred', linestyle='--',label='Model DLWRF')
        
        ax.plot(time, rn_obs_rad, color='plum', label='Observed RN')
        ax.plot(time, rn_mod_rad, color='plum', linestyle='--', label='Model RN')
        
        ax.set_xticks(time[::3])
        ax.tick_params(axis='x', labelsize=12, rotation=25)
        ax.tick_params(axis='y', labelsize=12)
        
        legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
        legend.get_frame().set_edgecolor('black')
        legend.get_frame().set_linewidth(0.8)
        legend.get_frame().set_facecolor('snow')
        
        ax.set_ylabel('W m$^{-2}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
        ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
        ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet Radiative Energy Budget\n{time.iloc[0]}Z to {time.iloc[-1]}Z at {site}', fontsize=18, fontweight='bold')
        
        ax.grid(True, alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'{savepath}RAD_{site}.png', dpi=300)

## COMPUTE AVERAGE ACROSS SITES SELECTED

In [ ]:
df_temp_group = df_temp.groupby(['day','hour']).mean(numeric_only=True)
df_temp_group.reset_index(inplace=True)
df_model_group = model_df.groupby(['forecast_hour']).mean(numeric_only=True)
df_model_group.reset_index(inplace=True)
df_stand_group = df_temp2.groupby(['day','hour']).mean(numeric_only=True)
df_stand_group.reset_index(inplace=True)
df_stand_group['datetime']=df_stand_group['day'].astype(str)+ ' ' + df_stand_group['hour'].astype(str) + ':00Z'
df_model_group['datetime']=df_stand_group['datetime']
df_model_group

In [ ]:
name = 'rad'
time = df_model_group['datetime'].copy()

### PLOTTING ###
if name == 'wind':
    # Plot
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df_model_group['datetime'], df_stand_group['avg_wind_speed_merge [m/s]'], color='darkslategrey', label='Observed Wind Speed')
    ax.plot(df_model_group['datetime'], df_model_group['WIND'], color='darkslategrey', linestyle='--', label='Model Wind Speed')
    average_obs = np.mean(df_stand_group['avg_wind_speed_merge [m/s]'])
    average_model = np.mean(df_model_group['WIND'])
    average_diff = np.mean(df_stand_group['avg_wind_speed_merge [m/s]'] - df_model_group['WIND'])
    ax.scatter([], [], marker = '.', color = 'k', label=f'Average Observed Wind Speed = {average_obs:.2f} m s$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'k', label=f'Average Model Wind Speed = {average_model:.2f} m s$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'k', label=f'Average Obs. - Model Wind Speed = {average_diff:.2f} m s$^{{-1}}$')

    ax.set_xticks(time[::3])
    ax.tick_params(axis='x', labelsize=12, rotation=25)
    ax.tick_params(axis='y', labelsize=12)
    
    legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
    legend.get_frame().set_edgecolor('black')
    legend.get_frame().set_linewidth(0.8)
    legend.get_frame().set_facecolor('snow')
    
    ax.set_ylabel('m s$^{-1}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
    ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
    ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet Wind Speed \n{time.iloc[0]}Z to {time.iloc[-1]}  \n Averaged Across OWEG, PENN, CLAR, ONTA, and WARS', fontsize=18, fontweight='bold')
    
    ax.grid(True, alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'{savepath}Average_WIND_{site}.png', dpi=300)
    #plt.close()

elif name=='temp':
    # Plot
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df_model_group['datetime'], df_stand_group['temp_2m [degC]'], color='firebrick', label='Observed TMP')
    ax.plot(df_model_group['datetime'], df_model_group['TMP'], color='firebrick', linestyle='--', label='Model TMP')
    ax.plot(df_model_group['datetime'],  df_stand_group['dewpoint [degC]'], color='green', label='Observed DPT')
    ax.plot(df_model_group['datetime'], df_model_group['DPT'], color='green', linestyle='--', label='Model DPT')

    average_obs_T = np.mean(df_stand_group['temp_2m [degC]'])
    average_model_T = np.mean(df_model_group['TMP'])
    average_diff_T = np.mean(df_stand_group['temp_2m [degC]'] - df_model_group['TMP'])
    ax.scatter([], [], marker = '.', color = 'firebrick', label=f'Average Observed Temperature = {average_obs_T:.2f} $^{{o}}C$')
    ax.scatter([], [], marker = '.', color = 'firebrick', label=f'Average Model Temperature = {average_model_T:.2f} $^{{o}}C$')
    ax.scatter([], [], marker = '.', color = 'firebrick', label=f'Average Obs. - Model Temperature = {average_diff_T:.2f} $^{{o}}C$')

    average_obs_Td = np.mean(df_stand_group['dewpoint [degC]'])
    average_model_Td = np.mean(df_model_group['DPT'])
    average_diff_Td = np.mean(df_stand_group['dewpoint [degC]'] - df_model_group['DPT'])
    ax.scatter([], [], marker = '.', color = 'g', label=f'Average Observed Dewpoint = {average_obs_Td:.2f} $^{{o}}C$')
    ax.scatter([], [], marker = '.', color = 'g', label=f'Average Model Dewpoint = {average_model_Td:.2f} $^{{o}}C$')
    ax.scatter([], [], marker = '.', color = 'g', label=f'Average Obs. - Model Dewpoint = {average_diff_Td:.2f} $^{{o}}C$')
    
    
    ax.set_xticks(time[::3])
    ax.tick_params(axis='x', labelsize=12, rotation=25)
    ax.tick_params(axis='y', labelsize=12)
    
    legend = ax.legend(loc='center left', fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
    legend.get_frame().set_edgecolor('black')
    legend.get_frame().set_linewidth(0.8)
    legend.get_frame().set_facecolor('snow')
    
    ax.set_ylabel('$^{o}$C', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
    ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
    ax.set_title(f'Average HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet \n 2-meter Temperature & Dewpoint Temperatures \n{time.iloc[0]}Z to {time.iloc[-1]} \n Averaged Across OWEG, PENN, CLAR, ONTA, and WARS', fontsize=18, fontweight='bold')
    
    ax.grid(True, alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'{savepath}Average_TMPDPT.png', dpi=300)

elif name =='sfc':
    # Plot
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df_model_group['datetime'], df_temp_group['H'], color='red', label='Observed H')
    ax.plot(df_model_group['datetime'], df_temp_group['HL'], color='navy', label='Observed HL')
    ax.plot(df_model_group['datetime'], df_temp_group['HG'], color='limegreen', label='Observed HG')
    ax.plot(df_model_group['datetime'], df_temp_group['Rn'], color='plum', label='Observed RN')
    ax.plot(df_model_group['datetime'], df_model_group['SHTFL'], color='red', linestyle='--', label='Model H')
    ax.plot(df_model_group['datetime'], df_model_group['LHTFL'], color='navy', linestyle='--', label='Model HL')
    ax.plot(df_model_group['datetime'], df_model_group['GFLUX'], color='limegreen', linestyle='--', label='Model HG')
    ax.plot(df_model_group['datetime'], df_model_group['RN_SFCBUDGET'], color='plum', linestyle='--', label='Model RN')

    average_obs_H = np.mean(df_temp_group['H'])
    average_model_H = np.mean(df_model_group['SHTFL'])
    average_diff_H = np.mean(df_temp_group['H'] - df_model_group['SHTFL'])
    ax.scatter([], [], marker = '.', color = 'red', label=f'Average Observed H = {average_obs_H:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'red', label=f'Average Model H = {average_model_H:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'red', label=f'Average Obs. - Model H = {average_diff_H:.2f} W m$^{{-1}}$')

    average_obs_HL = np.mean(df_temp_group['HL'])
    average_model_HL = np.mean(df_model_group['LHTFL'])
    average_diff_HL = np.mean(df_temp_group['HL'] - df_model_group['LHTFL'])
    ax.scatter([], [], marker = '.', color = 'navy', label=f'Average Observed HL = {average_obs_HL:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'navy', label=f'Average Model HL = {average_model_HL:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'navy', label=f'Average Obs. - Model HL = {average_diff_HL:.2f} W m$^{{-1}}$')
    
    average_obs_HG = np.mean(df_temp_group['HG'])
    average_model_HG = np.mean(df_model_group['GFLUX'])
    average_diff_HG = np.mean(df_temp_group['HG'] - df_model_group['GFLUX'])
    ax.scatter([], [], marker = '.', color = 'limegreen', label=f'Average Observed HG = {average_obs_HG:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'limegreen', label=f'Average Model HG = {average_model_HG:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'limegreen', label=f'Average Obs. - Model HG = {average_diff_HG:.2f} W m$^{{-1}}$')

    average_obs_RN = np.mean(df_temp_group['Rn'])
    average_model_RN = np.mean(df_model_group['RN_SFCBUDGET'])
    average_diff_RN = np.mean(df_temp_group['Rn'] - df_model_group['RN_SFCBUDGET'])
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Observed RN = {average_obs_RN:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Model RN = {average_model_RN:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Obs. - Model RN = {average_diff_RN:.2f} W m$^{{-1}}$')
    

    
    ax.set_xticks(time[::3])
    ax.tick_params(axis='x', labelsize=12, rotation=25)
    ax.tick_params(axis='y', labelsize=12)
    
    legend = ax.legend(loc='center left', ncol=2, fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
    legend.get_frame().set_edgecolor('black')
    legend.get_frame().set_linewidth(0.8)
    legend.get_frame().set_facecolor('snow')
    
    ax.set_ylabel('W m$^{-2}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
    ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
    ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet Surface Energy Budget\n{time.iloc[0]}Z to {time.iloc[-1]} \n Averaged Across OWEG, PENN, CLAR, ONTA, and WARS', fontsize=18, fontweight='bold')
    
    ax.grid(True, alpha=0.7)
    #plt.tight_layout()
    plt.savefig(f'{savepath}Average_SFCBUD.png', dpi=300)

elif name=='rad':
    # Plot
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df_model_group['datetime'], df_temp_group['SW_OUT'], color='forestgreen', label='Observed USWRF')
    ax.plot(df_model_group['datetime'], df_model_group['USWRF'], color='forestgreen', linestyle='--', label='Model USWRF')
    ax.plot(df_model_group['datetime'], df_temp_group['SW_IN'], color='darkorange', label='Observed DSWRF')
    ax.plot(df_model_group['datetime'], df_model_group['DSWRF'], color='darkorange', linestyle='--',label='Model DSWRF')
    
    ax.plot(df_model_group['datetime'], df_temp_group['LW_OUT'], color='dodgerblue', label='Observed ULWRF')
    ax.plot(df_model_group['datetime'], df_model_group['ULWRF'], color='dodgerblue', linestyle='--', label='Model ULWRF')
    ax.plot(df_model_group['datetime'], df_temp_group['LW_IN'], color='mediumvioletred', label='Observed DLWRF')
    ax.plot(df_model_group['datetime'], df_model_group['DLWRF'], color='mediumvioletred', linestyle='--',label='Model DLWRF')
    ax.plot(df_model_group['datetime'], df_temp_group['Rn'], color='plum', label='Observed RN')
    ax.plot(df_model_group['datetime'], df_model_group['RN_RAD'], color='plum', linestyle='--', label='Model RN')


    average_obs_USWRF = np.mean(df_temp_group['SW_OUT'])
    average_model_USWRF = np.mean(df_model_group['USWRF'])
    average_diff_USWRF = np.mean(df_temp_group['SW_OUT'] - df_model_group['USWRF'])
    ax.scatter([], [], marker = '.', color = 'forestgreen', label=f'Average Observed USWRF = {average_obs_USWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'forestgreen', label=f'Average Model USWRF = {average_model_USWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'forestgreen', label=f'Average Obs. - Model USWRF = {average_diff_USWRF:.2f} W m$^{{-1}}$')

    average_obs_DSWRF = np.mean(df_temp_group['SW_IN'])
    average_model_DSWRF = np.mean(df_model_group['DSWRF'])
    average_diff_DSWRF = np.mean(df_temp_group['SW_IN'] - df_model_group['DSWRF'])
    ax.scatter([], [], marker = '.', color = 'darkorange', label=f'Average Observed DSWRF = {average_obs_DSWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'darkorange', label=f'Average Model DSWRF = {average_model_DSWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'darkorange', label=f'Average Obs. - Model DSWRF = {average_diff_DSWRF:.2f} W m$^{{-1}}$')
    
    average_obs_ULWRF = np.mean(df_temp_group['LW_OUT'])
    average_model_ULWRF = np.mean(df_model_group['ULWRF'])
    average_diff_ULWRF = np.mean(df_temp_group['LW_OUT'] - df_model_group['ULWRF'])
    ax.scatter([], [], marker = '.', color = 'dodgerblue', label=f'Average Observed ULWRF = {average_obs_ULWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'dodgerblue', label=f'Average Model ULWRF = {average_model_ULWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'dodgerblue', label=f'Average Obs. - Model ULWRF = {average_diff_ULWRF:.2f} W m$^{{-1}}$')

    average_obs_DLWRF = np.mean(df_temp_group['LW_IN'])
    average_model_DLWRF = np.mean(df_model_group['DLWRF'])
    average_diff_DLWRF = np.mean(df_temp_group['LW_IN'] - df_model_group['DLWRF'])
    ax.scatter([], [], marker = '.', color = 'mediumvioletred', label=f'Average Observed DLWRF = {average_obs_DLWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'mediumvioletred', label=f'Average Model DLWRF = {average_model_DLWRF:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'mediumvioletred', label=f'Average Obs. - Model DLWRF = {average_diff_DLWRF:.2f} W m$^{{-1}}$')

    
    average_obs_RN = np.mean(df_temp_group['Rn'])
    average_model_RN = np.mean(df_model_group['RN_RAD'])
    average_diff_RN = np.mean(df_temp_group['Rn'] - df_model_group['RN_RAD'])
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Observed RN = {average_obs_RN:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Model RN = {average_model_RN:.2f} W m$^{{-1}}$')
    ax.scatter([], [], marker = '.', color = 'plum', label=f'Average Obs. - Model RN = {average_diff_RN:.2f} W m$^{{-1}}$')
        
    ax.set_xticks(time[::3])
    ax.tick_params(axis='x', labelsize=12, rotation=25)
    ax.tick_params(axis='y', labelsize=12)
    
    legend = ax.legend(loc='center left', ncol=2, fontsize=12, bbox_to_anchor=(1, 0.5), frameon=True)
    legend.get_frame().set_edgecolor('black')
    legend.get_frame().set_linewidth(0.8)
    legend.get_frame().set_facecolor('snow')
    
    ax.set_ylabel('W m$^{-2}$', fontsize=15, rotation='horizontal', labelpad=35, fontweight='bold')
    ax.set_xlabel('Time', fontsize=15, labelpad=10, fontweight='bold')
    ax.set_title(f'HRRR F{forecast_hrs[0]}-F{forecast_hrs[-1]} Model vs Observed NYS Mesonet Radiative Energy Budget\n{time.iloc[0]}Z to {time.iloc[-1]}\n Averaged Across OWEG, PENN, CLAR, ONTA, and WARS', fontsize=18, fontweight='bold')
    
    ax.grid(True, alpha=0.7)
    #plt.tight_layout()
    plt.savefig(f'{savepath}Average_RAD.png', dpi=300)

In [ ]:
# read in profiler data
df=pd.read_csv(r'/home/jovyan/Desktop/Micrometeorology/Profiler data/NYSMesonet_3D_2023-06_06-07.csv')# 20230606-3d
df['datetime']=pd.to_datetime(df['time'])
df.columns

In [ ]:
# map lats and lons to sites
lat_vals=[42.75175, 43.78823, 40.87248, 42.99359, 44.889, 42.01243, 40.97039, 43.06874,
         42.02493, 40.73433, 42.99983, 40.60401, 40.91957, 41.13303, 44.22425, 40.65025, 43.2601]
lon_vals=[-73.81128, -76.11765, -73.89352, -78.79461, -73.46634, -79.62746, -72.20094, -76.46999,
         -76.25307, -73.81585, -73.88412, -74.14849, -73.13328, -74.08597,-74.44105, -73.5054, -77.41238]
lats = {
    'PROF_ALBA': lat_vals[0],
    'PROF_BELL': lat_vals[1],
    'PROF_BRON': lat_vals[2],
    'PROF_BUFF': lat_vals[3],
    'PROF_CHAZ': lat_vals[4],
    'PROF_CLYM': lat_vals[5],
    'PROF_EHAM': lat_vals[6],
    'PROF_JORD': lat_vals[7],
    'PROF_OWEG': lat_vals[8],
    'PROF_QUEE': lat_vals[9],
    'PROF_REDH': lat_vals[10],
    'PROF_STAT': lat_vals[11],
    'PROF_STON': lat_vals[12],
    'PROF_SUFF': lat_vals[13],
    'PROF_TUPP': lat_vals[14],
    'PROF_WANT': lat_vals[15],
    'PROF_WEBS': lat_vals[16]
}
lons = {
    'PROF_ALBA': lon_vals[0],
    'PROF_BELL': lon_vals[1],
    'PROF_BRON': lon_vals[2],
    'PROF_BUFF': lon_vals[3],
    'PROF_CHAZ': lon_vals[4],
    'PROF_CLYM': lon_vals[5],
    'PROF_EHAM': lon_vals[6],
    'PROF_JORD': lon_vals[7],
    'PROF_OWEG': lon_vals[8],
    'PROF_QUEE': lon_vals[9],
    'PROF_REDH': lon_vals[10],
    'PROF_STAT': lon_vals[11],
    'PROF_STON': lon_vals[12],
    'PROF_SUFF': lon_vals[13],
    'PROF_TUPP': lon_vals[14],
    'PROF_WANT': lon_vals[15],
    'PROF_WEBS': lon_vals[16]
}
df['latitude'] = df['station'].map(lats)
df['longitude'] = df['station'].map(lons)

In [ ]:
# PLOT ALL SITES
fig=plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='-')
ax.add_feature(cfeature.LAND, facecolor='black',alpha=0.3)
ax.add_feature(cfeature.LAKES, facecolor='lightblue', alpha=0.3)

sites=df['station'].unique()
cmap = cm.get_cmap('inferno', len(sites))

for i,site in enumerate(sites):
    lon=df.loc[df['station'] == site]['longitude'].unique()
    lat=df.loc[df['station']== site]['latitude'].unique()
    ax.plot(lon, lat, 's',markersize=10, color=cmap(i), label=f'{site[5::]}')
    ax.text(lon-0.15,lat-0.3,f'{site[5::]}', rotation=25, color=cmap(i),fontweight='bold', fontsize=12)

gl = ax.gridlines(draw_labels=True,linewidth=1, color='black', alpha=0.7, linestyle='--')
gl.top_labels = False
ax.set_extent([-80, -71, 40, 45.4])  
ax.legend(loc='upper right')
ax.set_title('Plot of NY Profiler Sites', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('PART3_sites.png',dpi=300)

In [ ]:
# investigate OWEG site
df=df.loc[df['station'] == 'PROF_OWEG']
df=df.loc[(df['time'] >= '2023-06-06 18:00:00 UTC') & (df['time'] <= '2023-06-07 18:00:00 UTC')]
df

In [ ]:
# set variable of interest
variable_list = ['vapor_density [g/m^3]', 'temperature [K]', 'relative_humidity [%]', 'liquid [g/m^3]', 'cnr [dB]']
variable_str_list = ['Vapor Density (g m$^{-3}$)', 'Temperature (K)', 'Relative Humidity (%)', 'Liquid (g m$^{-3}$)', 'Carrier-to-Noise (dB)']
colors = ['magma', 'inferno', 'YlGn', 'Blues', 'ChaseSpectral']

num_p = 4
variable=variable_list[num_p]
variable_str=variable_str_list[num_p]
color=colors[num_p]

# create a new dataframe to reshape the grid. if you try to run the code with a regular pcolormesh(t,h,var) with no pivot, you'll get an error (less variables than expected)
df_pivot = df.pivot(index='range [m]', columns='time', values=variable) # index is now the range values and not time. time becomes the columns populating the df. the values in each column correspond to 
                                                                        # variable values at each range value for each time
df_pivot

In [ ]:
# set values/objects
h = df_pivot.index.values # since h is the index now, call .index to get the values
t = df_pivot.columns.values # call the time values since they're now populating the df's columns
var = df_pivot.values # call the values of the table. they'll populate col x row of the array based on their position in the df
site=df['station'].unique()[0]
site=site[5::]

# plot data
fig=plt.figure(figsize=(20,9))
mesh = plt.pcolormesh(t, h, var, cmap=color)
cbar=plt.colorbar(mesh, label=variable_str, orientation='horizontal', pad=0.21)
cbar.set_label(f'{variable_str}', fontsize = 15, fontweight = 'bold')
cbar.ax.tick_params(labelsize=14)

plt.xticks(t[::11],rotation=25)
plt.tick_params(axis='both', labelsize=13)
plt.ylabel('Height (m)', fontweight='bold', fontsize=15, rotation='horizontal', labelpad=55)
plt.xlabel('Time (UTC)', fontweight='bold', fontsize=15)
plt.title(f'{variable_str} at {site} NYS Mesonet Profiler Site from 18:05 UTC 6 June - 17:55 UTC 7 June 2023', fontweight='bold',fontsize=22, pad=15)
plt.tight_layout()
plt.savefig(f'/home/jovyan/Desktop/Micrometeorology/Profiler data/PLOTS/0607-2023_june_{variable_str}.png',dpi=300)

In [ ]:
# CHANGE THE YLIM -- no data above 7000 m
# set values/objects
h = df_pivot.index.values # since h is the index now, call .index to get the values
t = df_pivot.columns.values # call the time values since they're now populating the df's columns
var = df_pivot.values # call the values of the table. they'll populate col x row of the array based on their position in the df
site=df['station'].unique()[0]
site=site[5::]

# plot data
fig=plt.figure(figsize=(20,9))
mesh = plt.pcolormesh(t, h, var, cmap=color)
cbar=plt.colorbar(mesh, label=variable_str, orientation='horizontal', pad=0.25)
cbar.set_label(f'{variable_str}', fontsize = 15, fontweight = 'bold')
cbar.ax.tick_params(labelsize=14)

plt.xticks(t[::11],rotation=25)
plt.ylim(0,7000)
plt.tick_params(axis='both', labelsize=13)
plt.ylabel('Height (m)', fontweight='bold', fontsize=15, rotation='horizontal', labelpad=55)
plt.xlabel('Time (UTC)', fontweight='bold', fontsize=15)
plt.title(f'{variable_str} at {site} NYS Mesonet Profiler Site from 18:05 UTC 6 June 2023 - 17:55 UTC 7 June 2023', fontweight='bold',fontsize=22, pad=15)
plt.savefig(f'/home/jovyan/Desktop/Micrometeorology/Profiler data/PLOTS/0607-2023_june_{variable_str}.png',dpi=300)